In [5]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

In [91]:
batch_size = 8
lr = 1e-4
epochs = 25
height = 512
width = 512

In [87]:
model_file = os.path.join("/Users/patrickobrien/Desktop/UNET/", "lung_UNET.keras")

In [67]:
def conv_block(input, num_filters):
  x = Conv2D(num_filters, 3, padding="same")(input)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  x = Conv2D(num_filters, 3, padding="same")(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  return x

def encoder_block(input, num_filters):
  x = conv_block(input, num_filters)
  p = MaxPool2D((2, 2))(x)
  return x, p

def decoder_block(input, skip_features, num_filters):
  x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
  x = Concatenate()([x, skip_features])
  x = conv_block(x, num_filters)
  return x

In [70]:
def build_unet(input_shape):
  inputs = Input(input_shape)

  s1, p1 = encoder_block(inputs, 64)
  s2, p2 = encoder_block(p1, 128)
  s3, p3 = encoder_block(p2, 256)
  s4, p4 = encoder_block(p3, 512)

  b1 = conv_block(p4, 1024)

  d1 = decoder_block(b1, s4, 512)
  d2 = decoder_block(d1, s3, 256)
  d3 = decoder_block(d2, s2, 128)
  d4 = decoder_block(d3, s1, 64)

  outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

  model = Model(inputs, outputs, name="U-Net")
  return model

In [77]:
path = '/Users/patrickobrien/Desktop/UNET/'

def load_data(path):
  train_y = sorted(glob(os.path.join(path,'mask', '*')))
  train_x = sorted(glob(os.path.join(path,'image', '*')))

  valid_y = sorted(glob(os.path.join(path,'mask_val', '*')))
  valid_x = sorted(glob(os.path.join(path,'image_val', '*')))

  return (train_x, train_y), (valid_x, valid_y)

In [72]:
def read_image(path):
  path = path.decode()
  x = cv2.imread(path)
  x = x/255.0
  return x

In [73]:
def read_mask(path):
  path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  x = x/255.0
  x = np.expand_dims(x, axis=-1)
  return x

In [74]:
def tf_parse(x, y):
  def _parse(x, y):
    x = read_image(x)
    y = read_mask(y)
    return x, y
  result = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
  x = result[0]
  y = result[1]
  x.set_shape([height, width, 3])
  y.set_shape([height, width, 1])
  return x, y

In [75]:
def tf_dataset(x, y, batch=8):
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
  dataset = dataset.batch(batch)
  dataset = dataset.prefetch(tf.data.AUTOTUNE)
  return dataset

In [81]:
(train_x, train_y), (valid_x, valid_y) = load_data(path)
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [83]:
input_shape = (height, width, 3)
model = build_unet(input_shape)

In [ ]:
model.summary()

In [85]:
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["acc"])

In [90]:
callbacks = [
    ModelCheckpoint(model_file, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
]

In [92]:
model.fit(train_dataset, validation_data=valid_dataset, epochs=epochs, callbacks=callbacks)

Epoch 1/25
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 34s/step - acc: 0.9060 - loss: 0.2847 
Epoch 1: val_loss improved from inf to 0.59293, saving model to /Users/patrickobrien/Desktop/UNET/my_model.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1988s 36s/step - acc: 0.9065 - loss: 0.2831 - val_acc: 0.7550 - val_loss: 0.5929 - learning_rate: 1.0000e-04
Epoch 2/25
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 34s/step - acc: 0.9587 - loss: 0.1138 
Epoch 2: val_loss improved from 0.59293 to 0.45182, saving model to /Users/patrickobrien/Desktop/UNET/my_model.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1962s 36s/step - acc: 0.9587 - loss: 0.1139 - val_acc: 0.8975 - val_loss: 0.4518 - learning_rate: 1.0000e-04
Epoch 3/25
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 31s/step - acc: 0.9559 - loss: 0.1096 
Epoch 3: val_loss did not improve from 0.45182
55/55 ━━━━━━━━━━━━━━━━━━━━ 1831s 33s/step - acc: 0.9559 - loss: 0.1095 - val_acc: 0.6365 - val_loss: 1.2037 - learning_rate: 1.0000e-04
Epoch 4/25
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 355s/step - acc: 0.9541 - loss: 0.

In [93]:
model.save('/Users/patrickobrien/Desktop/UNET/lung_UNET.keras')

In [94]:
trained_model = tf.keras.models.load_model('/Users/patrickobrien/Desktop/UNET/lung_UNET.keras')

In [95]:
trained_model.summary()

Model: "U-Net"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 512, 512,  │      1,792 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512, 512,  │        256 │ conv2d_19[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_18       │ (None, 512, 512,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 512, 512,  │     36,928 │ activation_18[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512, 512,  │        256 │ conv2d_20[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_19       │ (None, 512, 512,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 256, 256,  │          0 │ activation_19[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 256, 256,  │     73,856 │ max_pooling2d_4[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        512 │ conv2d_21[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_20       │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 256, 256,  │    147,584 │ activation_20[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        512 │ conv2d_22[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_21       │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 128, 128,  │          0 │ activation_21[0]… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 128, 128,  │    295,168 │ max_pooling2d_5[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │      1,024 │ conv2d_23[0][0] 

 Total params: 93,142,341 (355.31 MB)

 Trainable params: 31,043,521 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)

 Optimizer params: 62,087,044 (236.84 MB)